# MultiAgent Pattern

Multiple agents collaborate to complete a complex task in a structured workflow.

<img src = "https://github.com/neural-maze/agentic_patterns/raw/main/img/multiagent_pattern.png" height ="650">

In [62]:
from agentic_patterns.multiagent_pattern.agent import Agent

model = "llama-3.3-70b-versatile"

agent_1 = Agent(
        name="Code Generator Agent",
        backstory="You are an expert software engineer, skilled in writing clean and efficient code.",
        task_description="Write a Python function that calculates the factorial of a number.",
        task_expected_output="Just output the Python code without any explanations or comments.",
        llm=model
    )

print(agent_1.run())

```python
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

def factorial_iterative(n):
    result = 1
    for i in range(1, n+1):
        result *= i
    return result
```


In [63]:
from agentic_patterns.tool_pattern.tool import tool
@tool
def write_str_to_text(string_data: str, txt_filename: str):
    with open(txt_filename, mode='w', encoding='utf-8') as file:
        file.write(string_data)

    print(f"Data successfully written to {txt_filename}")

In [64]:
agent_tool_example = Agent(
    name="Writer Agent",
    backstory="You are a language model specialised in writing text into .txt files",
    task_description="Write the string 'This is a Tool Agent' into './tool_agent_example.txt'",
    task_expected_output="A .txt file containing the given string",
    tools=write_str_to_text,
    llm = model
)


In [65]:
agent_tool_example.run()

"A .txt file containing the string 'This is a Tool Agent' has been created."

## Agent Dependencies

In [66]:
agent_1 = Agent(
        name="Code Generator Agent",
        backstory="You are an expert software engineer, skilled in writing clean and efficient code.",
        task_description="Write a Python function that calculates the factorial of a number.",
        task_expected_output="Just output the Python code without any explanations or comments.",
        llm=model
    )

agent_2 = Agent(
        name="Code Translator Agent",
        backstory="You are a programming polyglot, skilled in translating code into multiple languages.",
        task_description="Translate a Python function into JavaScript.", 
        task_expected_output="Just output the JavaScript code and nothing else.",
        llm=model
    )

In [67]:
#We can define the agent dependencies using the `>>` operator.
agent_1 >> agent_2

Code Translator Agent

In [68]:
print(f"Agent 1 dependencies: {agent_1.dependencies}")
print(f"Agent 1 dependents: {agent_1.dependents}")
print(f"Agent 2 dependencies:  {agent_2.dependencies}")
print(f"Agent 2 dependents:  {agent_2.dependents}")

Agent 1 dependencies: []
Agent 1 dependents: [Code Translator Agent]
Agent 2 dependencies:  [Code Generator Agent]
Agent 2 dependents:  []


In [69]:
print(agent_1.run())

def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)


In [70]:
print(agent_2.context)

Code Translator Agent received context: 
def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)


# Crew

In [71]:
from agentic_patterns.multiagent_pattern.crew import Crew

with Crew() as crew:
    agent_1 = Agent(
        name="Code Generator Agent",
        backstory="You are an expert software engineer, skilled in writing clean and efficient code.",
        task_description="Write a Python function that calculates the factorial of a number.",
        task_expected_output="Just output the Python code without any explanations or comments.",
        llm=model
    )

    agent_2 = Agent(
        name="Code Translator Agent",
        backstory="You are a programming polyglot, skilled in translating code into multiple languages.",
        task_description="Translate a Python function into JavaScript.", 
        task_expected_output="Just output the JavaScript code and nothing else.",
        llm=model
    )

    agent_3 = Agent(
        name="File Writer Agent",
        backstory="You are an automation expert, specializing in saving code snippets into files.",
        task_description="You'll receive JavaScript code in your context. You need to write the code into './code.js' file.",
        task_expected_output="A JavaScript file containing the translated code received from the context.",
        tools=write_str_to_text,
        llm=model
    )

    agent_1 >> agent_2 >> agent_3


In [72]:
print(crew.run())


RUNNING AGENT: Code Generator Agent

def factorial(n):
    if n == 0:
        return 1
    else:
        return n * factorial(n-1)

RUNNING AGENT: Code Translator Agent

```javascript
function factorial(n) {
    if (n == 0) {
        return 1;
    } else {
        return n * factorial(n-1);
    }
}
```

RUNNING AGENT: File Writer Agent


Thought: I need to write the provided JavaScript code into a file named './code.js'. The code is provided in the context and I have a tool to write string data into a text file.

Using Tool: write_str_to_text

Tool call dict: 
{'name': 'write_str_to_text', 'arguments': {'string_data': 'function factorial(n) {\n    if (n == 0) {\n        return 1;\n    } else {\n        return n * factorial(n-1);\n    }\n}', 'txt_filename': './code.js'}, 'id': 0}
Data successfully written to ./code.js

Tool result: 
None

Observations: {0: None}
A JavaScript file containing the provided code has been created at './code.js'
None
